### Import libraries

In [1]:
import requests
import json
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time

from lxml import objectify

from bibliopixel import *
from bibliopixel.drivers.LPD8806 import *
from bibliopixel import LEDStrip
import bibliopixel.colors as colors

### Setting up LED

In [2]:
LedsPerSide = 8
numLeds= LedsPerSide*4*2 ##x/side * 4 sides * 2 levels
driver=DriverLPD8806(numLeds)
led=LEDStrip(driver)

###Extracting data from xml files

In [20]:
path = 'cc_customer_11.xml'
xml = objectify.parse(open(path))
root = xml.getroot()

In [21]:
root.getchildren() #Meter readings in root.getchildren[7]

['urn:uuid:e3f79426-1069-409e-a8eb-475528ed81e1',
 'Test GreenButton',
 '2013-04-04T03:35:38Z',
 u'',
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee488>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee4d0>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee518>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee560>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee5a8>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x10acee5f0>]

In [22]:
obj_top = root.getchildren()[7].getchildren()[4].getchildren()[0].getchildren()
length = len(obj_top)
length

105408

In [23]:
df = pd.DataFrame(columns=('duration', 'time_stamp', 'value'))

In [24]:
for i in range(1,length): #takes a while to run #element 0 is summary
    obj = obj_top[i].getchildren()
    row = dict(zip(['duration', 'time_stamp', 'value'], [np.int(obj[0].getchildren()[0]), np.int(obj[0].getchildren()[1]), np.int(obj[1])]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)

In [26]:
(df['value'].mean(), df['value'].min(), df['value'].max()) # value is in watt-hrs

(318232.83326534292, 85751.0, 527377.0)

In [27]:
df

,duration,time_stamp,value
1,300,1325376600,105789
2,300,1325376900,104731
3,300,1325377200,102292
4,300,1325377500,102785
5,300,1325377800,101610
6,300,1325378100,100643
7,300,1325378400,101130
8,300,1325378700,101967
9,300,1325379000,101454
10,300,1325379300,99754


### Adding timestamp and grouping

In [30]:
start_time = pd.to_datetime('2011-12-31 20:10') 
## Got this from the sheet: 
## Link: http://services.greenbuttondata.org/sample-data/cc_customer_11.xml

Timestamp('2011-12-31 20:10:00')

In [73]:
df['timestamp'] = pd.date_range(start_time, periods=105407, freq='5min')
df['year'] = pd.DatetimeIndex(df['timestamp']).year
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df['date'] = pd.DatetimeIndex(df['timestamp']).date

df_monthly = df.groupby(['month','year'])
df_monthly_total = df_monthly['value'].sum() #Monthly total usage
df_daily = df.groupby(['date'])
df_daily_total = df_daily['value'].sum()
df_daily_total


KeyError: 'Column not found: 0'

In [101]:
groups = dict(list(df_daily))


NameError: name 'datetime' is not defined

### Generating color scale

In [28]:
red = 0
green = 255
stepSize = 25
color_gn_rd = []
color_gn_rd.append((green, red, 0)) ## the lights are GRB format

In [69]:
while(red < 255): ## start with green and increase red
    red += stepSize;
    if(red > 245):
        red = 255; 
    color_gn_rd.append((green,red,0))

while(green > 0): ## start with red + green and decrease green
    green -= stepSize;
    if(green < 6):
        green = 0; 
    color_gn_rd.append((green,red,0)); 
    
len(color_gn_rd)

21

### Defining Flashing modes

In [67]:
intensity = np.arange(0,1.1,0.1)
reversed(intensity)
color = (50,100,110)
color_new = (color[0]*.9, color[1]*0.9,color[2]*0.9)
color_new

(45.0, 90.0, 99.0)

In [68]:
def led_set(start_position, numLEDs, color): ## Fills the colors
    led.fill(color, start=start_position,end=start_position+numLEDs)
    led.update()
    return

def pulse(start_position, numLEDs, color):
    ## Step-up intensity by 10% increments, then step down by the same every 0.1 seconds. total time = 4 sec
    
    intensity = np.arange(0,1.1,0.1) 

    for i in intensity:
        color_new = (color[0]*i,color[1]*i,color[3]*i) 
        # There is probably a more elegant way to do this.. 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)

    for i in reversed(intensity):
        color_new = (color[0]*i,color[1]*i,color[3]*i) 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)        

In [70]:
print ('Starting Display')
print ('Press \'control + c\' to stop' )


Starting Display
Press 'control + c' to stop


### Display 1: Last 10 day average on top, current usage on bottom

In [ ]:
i = 11 
last_index = len(df_daily_total)
last_10day_average = []
color_top = []
color_bottom = []
while (i < last_index):
    last_10day_average.append(df_daily_total[i-11:i-1].mean())
    